### Chris Decker

### CS544 – Final Project

In [53]:
# Turn off warnings
options(warn=-1)

# Set working directory to path with CSV:
filename = "Hospital_Value-Based_Purchasing__HVBP____Clinical_Care_Domain_Scores.csv"
filepath = file.choose()
dir = substr(filepath, 1, nchar(filepath) - nchar(filename))
setwd(dir)

# Import Data:
cms <- read.csv("Hospital_Value-Based_Purchasing__HVBP____Clinical_Care_Domain_Scores.csv", 
    stringsAsFactors = FALSE)

# Preprocess data. Convert all percentage rates to numerical. Coerce to NA if not
# a number.
cmsnames <- names(cms)
numericcols <- cmsnames[grepl("MORT", cmsnames, fixed = T) & !(grepl("Points", cmsnames, 
    fixed = T) | grepl("Score", cmsnames, fixed = T))]
for (i in numericcols) {
    cms[[i]] = as.numeric(cms[[i]])
}

# Set dataframe view parameters
options(repr.matrix.max.cols = 50, repr.matrix.max.rows = 100)

# Load libraries
library(plotly)
library(sampling)

# Performance of Hospitals against Clinical Care metrics for Medicare Value-Based Purchasing Program

### Background:  
Medicare is the largest Healthcare Payor in the US. The Centers for Medicare & Medicaid Services (CMS) developed the Value-Based Purchasing program in order to incentivize positive patient outcome.
For hospitals, there are 4 categories for which performance is measured:
<BR><BR>
    
![Domain Table](domaintable.png)
Source: [Centers for Medicare & Medicaid Services - HOSPITAL VALUE-BASED PURCHASING ](https://www.cms.gov/Outreach-and-Education/Medicare-Learning-Network-MLN/MLNProducts/downloads/Hospital_VBPurchasing_Fact_Sheet_ICN907664.pdf)
    
The following analyses will be focusing on the Clinical Care aspect of this program. In order to quantify Clinical Care hospital performance, CMS measures mortality rates for patients being treated for the following 3 conditions:
- Acute Myocardial Infarction (heart attack)
- Heart Failure
- Pneumonia  

Essentially, the less likely it is for a patient to die when visiting a given hospital with one of these conditions, the better the hospital is compensated for all Medicare-reimbursed visits.


### How it works:  
For each hospital, a baseline rate is established in a prior period. The hospital is then scored based on its performance against this baseline, the national benchmark rate, and the national achievement threshold.
<BR><BR>
    
![Rates](rates.png)

In [72]:
# Example data
head(cms[c("Hospital.Name", "MORT.30.AMI.Achievement.Threshold", "MORT.30.AMI.Benchmark", 
    "MORT.30.AMI.Baseline.Rate", "MORT.30.AMI.Performance.Rate")])

Hospital.Name,MORT.30.AMI.Achievement.Threshold,MORT.30.AMI.Benchmark,MORT.30.AMI.Baseline.Rate,MORT.30.AMI.Performance.Rate
MARSHALL MEDICAL CENTER SOUTH,0.850916,0.873053,0.836946,0.857280
DEKALB REGIONAL MEDICAL CENTER,0.850916,0.873053,0.840641,0.842346
CRESTWOOD MEDICAL CENTER,0.850916,0.873053,0.868536,0.874347
FORT DEFIANCE INDIAN HOSPITAL,0.850916,0.873053,0.850559,0.863356
TUBA CITY REGIONAL HEALTH CARE CORPORATION,0.850916,0.873053,0.853278,0.864334
CHINLE COMPREHENSIVE HEALTH CARE FACILITY,0.850916,0.873053,0.857041,0.868165


(Note: In CMS reporting, Mortality Rate reflects the number of survivors per patient, not deaths, so a higher rate reflects more positive patient outcomes.)
___

### Analysis 1 – Mortality Rate Correlation:
The first thing I was interested in was whether the mortality rates for the different conditions were correlated. That is, for a given hospital, if the mortality rate for (i.e.) Acute Myocardial Infarction was low, was there a tendency for the mortality rate for (i.e.) Pneumonia to be low as well. Since I was dealing with 3 different rates, the most appropriate way to visualize all of the relationships at once is with a pairwise plot:

In [73]:
# Pairwise plot
api_create(cms %>% plot_ly(width = 700, height = 600, marker = list(color = "rgba(0,0,0,0.25)", 
    line = list(color = "rgba(17, 157, 255, 0.5)", width = 1))) %>% add_trace(type = "splom", 
    dimensions = list(list(label = "AMI", values = ~MORT.30.AMI.Performance.Rate), 
        list(label = "HF", values = ~MORT.30.HF.Performance.Rate), list(label = "PN", 
            values = ~MORT.30.PN.Performance.Rate)), diagonal = list(visible = FALSE)) %>% 
    layout(title = "Mortality Rate Correlation", margin = list(l = 70, r = 50, b = 70, 
        t = 50)), filename = "pairwise")

Found a plot already named: 'pairwise'. Since fileopt='overwrite', I'll try to update it


<div>
    <a href="https://plot.ly/~djzero/0/?share_key=6rTI6X7CgtJ4wejb9AW0zI" target="_blank" title="pairwise" style="display: block; text-align: center;"><img src="https://plot.ly/~djzero/0.png?share_key=6rTI6X7CgtJ4wejb9AW0zI" alt="pairwise" style="max-width: 100%;width: 700px;"  width="700" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="djzero:0" sharekey-plotly="6rTI6X7CgtJ4wejb9AW0zI" src="https://plot.ly/embed.js" async></script>
</div>

Based on the lack of linearity in any of the plots, there appears to be no linear correlation between any one mortality rate with any other. This is reinforced by looking at the correlation rates, which show that no combination of mortality rates has a correlation higher than 37%:

In [75]:
# Correlation rates
cor(cms[c("MORT.30.AMI.Performance.Rate", "MORT.30.HF.Performance.Rate", "MORT.30.PN.Performance.Rate")], 
    use = "complete.obs")

,MORT.30.AMI.Performance.Rate,MORT.30.HF.Performance.Rate,MORT.30.PN.Performance.Rate
MORT.30.AMI.Performance.Rate,1.0000000,0.3029107,0.2930978
MORT.30.HF.Performance.Rate,0.3029107,1.0000000,0.3698133
MORT.30.PN.Performance.Rate,0.2930978,0.3698133,1.0000000


While there does not appear to be a correlation between rates, we can observe from the plot that the mortality rate for Pneumonia appears to be more favorable than for the other two conditions. 
___

###  Analysis 2 – Distribution of Mortality Rates
Stemming from the analysis on the correlation of different mortality rates, I’m interested in looking at their distribution and validating the observation that the mortality rate for Pneumonia is more favorable than the other two conditions. A boxplot is the best way to visualize this:

In [76]:
# Boxplots
api_create(stack(cms[c("MORT.30.AMI.Performance.Rate", "MORT.30.HF.Performance.Rate", "MORT.30.PN.Performance.Rate")]) %>% 
    plot_ly(y = ~values, x = ~ind, type = "box", width = 800, height = 600) %>% layout(title = "Mortality Rate Distribution", 
    margin = list(l = 70, r = 50, b = 70, t = 50), xaxis = list(title = "")), filename = "box")

<div>
    <a href="https://plot.ly/~djzero/2/?share_key=l1LEMNca5KsVdKcLNxq97T" target="_blank" title="box" style="display: block; text-align: center;"><img src="https://plot.ly/~djzero/2.png?share_key=l1LEMNca5KsVdKcLNxq97T" alt="box" style="max-width: 100%;width: 800px;"  width="800" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="djzero:2" sharekey-plotly="l1LEMNca5KsVdKcLNxq97T" src="https://plot.ly/embed.js" async></script>
</div>

We can see from the outliers that in certain hospitals, the mortality rate associated with Heart Failure is more favorable than the other two conditions, and that in other hospitals, the mortality rate associated with Pneumonia is less favorable than the other two conditions, however on average, Pneumonia is the condition that is least likely to result in patient death. For AMI and Heart Failure, the mean is roughly equidistant from the min and max values, suggesting a normal distribution.  For Pneumonia, the extreme outliers at the low end of the range suggest a left-skewed distribution. Histograms of the 3 distributions can serve to validate this observation:

In [79]:
# Histograms
ami <- cms %>% plot_ly(x = ~MORT.30.AMI.Performance.Rate, type = "histogram") %>% layout(annotations = list(text = "MORT.30.AMI.Performance.Rate", 
    xref = "paper", yref = "paper", yanchor = "bottom", xanchor = "center", align = "center", 
    x = 0.5, y = 1, showarrow = FALSE))
hf <- cms %>% plot_ly(x = ~MORT.30.HF.Performance.Rate, type = "histogram") %>% layout(annotations = list(text = "MORT.30.HF.Performance.Rate", 
    xref = "paper", yref = "paper", yanchor = "bottom", xanchor = "center", align = "center", 
    x = 0.5, y = 1, showarrow = FALSE))
pn <- cms %>% plot_ly(x = ~MORT.30.PN.Performance.Rate, type = "histogram") %>% layout(annotations = list(text = "MORT.30.PN.Performance.Rate", 
    xref = "paper", yref = "paper", yanchor = "bottom", xanchor = "center", align = "center", 
    x = 0.5, y = 1, showarrow = FALSE))
api_create(subplot(ami, hf, pn) %>% layout(title = "Mortality Rate Distributions", showlegend = F, 
    margin = list(l = 50, r = 50, b = 50, t = 70)), filename = "histograms")

<div>
    <a href="https://plot.ly/~djzero/4/?share_key=NjccH5w0SymegP5AnEA5Zx" target="_blank" title="histograms" style="display: block; text-align: center;"><img src="https://plot.ly/~djzero/4.png?share_key=NjccH5w0SymegP5AnEA5Zx" alt="histograms" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="djzero:4" sharekey-plotly="NjccH5w0SymegP5AnEA5Zx" src="https://plot.ly/embed.js" async></script>
</div>

___
### Analysis 3 – Mortality Rates by State
Now that I’ve established some understanding of the differences in mortality rate for each condition, I’m interested in seeing the differences by state. For this, the most appropriate visualization is a bar plot:

In [138]:
# Bar plot
statemean <- tapply(cms$MORT.30.AMI.Performance.Rate, list(cms$State), mean, na.rm = T)
stateheadtail <- c(head(sort(statemean), 10), tail(sort(statemean), 10))
statedf <- data.frame(State = names(stateheadtail), AvgAMIMort = stateheadtail)
statedf$State <- factor(statedf$State, levels = c(as.character(statedf$State)))
api_create(plot_ly(statedf, x = ~AvgAMIMort, y = ~State, type = "bar", width = 800, 
    height = 600) %>% layout(xaxis = list(range = c(0.84, 0.88)), title = "AMI Mortality Rate by State, Top/Bottom 10", 
    margin = list(l = 50, r = 50, b = 50, t = 70)), filename = "barplot")

<div>
    <a href="https://plot.ly/~djzero/6/?share_key=YhgdqGD4OihWmKqSYZTzom" target="_blank" title="barplot" style="display: block; text-align: center;"><img src="https://plot.ly/~djzero/6.png?share_key=YhgdqGD4OihWmKqSYZTzom" alt="barplot" style="max-width: 100%;width: 800px;"  width="800" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="djzero:6" sharekey-plotly="YhgdqGD4OihWmKqSYZTzom" src="https://plot.ly/embed.js" async></script>
</div>

The plot focuses on the mortality rate for Acute Myocardial Infarction as that is the most deadly condition. The results are somewhat in line with my preconceived notions that southern states would have less favorable mortality rates, as there are 4 in the bottom 10 (AR, MS, AL, TN) and none in the top 10, however I was surprised to see Washington in the bottom 10 as well. It was also interesting to observe that South Dakota was in the top 10 while North Dakota was in the bottom 10. With almost a point more favorable mortality rate than its nearest competitor, Rhode Island is statistically the best state in which to survive a heart attack.
___

### Analysis 4 – Distribution of Performance against Achievement, Benchmark, Baseline 
The mortality rates measured in the current period (‘Performance Rate’) are a means to an end; the hospitals are ultimately judged on how those rates compare to the Achievement Threshold, Benchmark Rate and Baseline Rate. I am interested in seeing the distribution of that comparison, again focusing on Acute Myocardial Infarction. In order to obtain a summary view, I assign a single string value denoting the performance against each. A pie plot can then be used to show the number of hospitals that fall into each summary bucket:


In [139]:
# Pie chart
comparison <- function(ach, bench, base, perf) {
    if (is.na(perf) | is.na(base) | is.na(ach) | is.na(bench)) 
        return(NA) else {
        if (perf - base > 0) {
            ba <- "H"
        } else if (perf - base < 0) {
            ba <- "L"
        } else {
            ba <- "S"
        }
        if (perf - ach > 0) {
            a <- "H"
        } else if (perf - ach < 0) {
            a <- "L"
        } else {
            a <- "S"
        }
        if (perf - bench > 0) {
            be <- "H"
        } else if (perf - bench < 0) {
            be <- "L"
        } else {
            be <- "S"
        }
        return(paste("Base:", ba, " Achmnt:", a, " Bench:", be, sep = ""))
    }
}
cms$AMIComp <- mapply(comparison, cms$MORT.30.AMI.Achievement.Threshold, cms$MORT.30.AMI.Benchmark, 
    cms$MORT.30.AMI.Baseline.Rate, cms$MORT.30.AMI.Performance.Rate)
api_create(plot_ly(stack(data.frame(rbind(table(cms$AMIComp)))), labels = ~ind, values = ~values, 
    type = "pie", width = 800, height = 600) %>% layout(title = "Distribution of AMI Performance Rate Comparison", 
    margin = list(l = 50, r = 50, b = 50, t = 70)), filename='pie')

<div>
    <a href="https://plot.ly/~djzero/8/?share_key=uOGjop6BDSXXG9zguRznIV" target="_blank" title="pie" style="display: block; text-align: center;"><img src="https://plot.ly/~djzero/8.png?share_key=uOGjop6BDSXXG9zguRznIV" alt="pie" style="max-width: 100%;width: 800px;"  width="800" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="djzero:8" sharekey-plotly="uOGjop6BDSXXG9zguRznIV" src="https://plot.ly/embed.js" async></script>
</div>

Since the Achievement Threshold was determined based on the 50th percentile of baseline values, the fact that greater than 50% of hospitals have a Performance Rate higher than the Achievment Threshold indicates an overall improvement across the population. Indeed, the biggest single group is of hospitals whose Performance Rate is higher than the Baseline Rate and the Achievment Threshold, but lower than the Benchmark Rate. The second-biggest group is higher than all 3. Since the Benchmark Rate was established based on the 90th percentile, the fact that more than 10% of hospitals have a Performance Rate higher than the Benchmark Rate indicates an improvement on the upper end of the spectrum as well. The smallest group of hospitals are those that exceeded the Achievement Threshold and Benchmark Rate, but were lower than the baseline rate, meaning they were high-performing hospitals that backtracked slightly.
___

### Applicability of Central Limit Theorem
In order to determine the extent to which the Central Limit Theorem applies to the AMI Mortality Rate, I will take the means of random samples of size 5 and 20, comparing the distribution of those sample means to the distribution of the population. For each sample size, I'll draw a number of means equal to the number of elements in the population.

In [47]:
# Random Samples & Overlaid Histogram
samples <- length(cms$MORT.30.AMI.Performance.Rate)
xbar5 <- numeric(samples)
for (i in 1:length(xbar5)) {
    s <- srswr(5, samples)
    rows <- (1:samples)[s != 0]
    xbar5[i] <- mean(cms$MORT.30.AMI.Performance.Rate[rows], na.rm = T)
}
xbar20 <- numeric(samples)
for (i in 1:length(xbar20)) {
    s <- srswr(20, samples)
    rows <- (1:samples)[s != 0]
    xbar20[i] <- mean(cms$MORT.30.AMI.Performance.Rate[rows], na.rm = T)
}
api_create(plot_ly(width = 800, height = 600) %>% add_histogram(x = ~cms$MORT.30.AMI.Performance.Rate, 
    name = "Population") %>% add_histogram(x = ~xbar5, name = "Sample Size 5") %>% 
    add_histogram(x = ~xbar20, name = "Sample Size 20") %>% layout(barmode = "overlay", 
    title = "Distribution of AMI Rate - Population vs. Sample Means", xaxis = list(title = ""), 
    margin = list(l = 50, r = 50, b = 50, t = 70)), filename = "sampleshist")

Found a grid already named: 'sampleshist Grid'. Since fileopt='overwrite', I'll try to update it
Found a plot already named: 'sampleshist'. Since fileopt='overwrite', I'll try to update it


<div>
    <a href="https://plot.ly/~djzero/10/?share_key=IhyQLFehxEcZm7bhDh64wQ" target="_blank" title="sampleshist" style="display: block; text-align: center;"><img src="https://plot.ly/~djzero/10.png?share_key=IhyQLFehxEcZm7bhDh64wQ" alt="sampleshist" style="max-width: 100%;width: 800px;"  width="800" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="djzero:10" sharekey-plotly="IhyQLFehxEcZm7bhDh64wQ" src="https://plot.ly/embed.js" async></script>
</div>
The overlaid histograms seem to show that the samples share a similar mean as the population, with standard deviation decreasing as the sample size increases. We can validate this by looking at the actual means and standard deviations from each set:

In [45]:
# Dataframe of Means & Standard Deviations
Set <- c("Population", "Sample Size 5", "Sample Size 20")
Mean <- c(mean(cms$MORT.30.AMI.Performance.Rate, na.rm = T), mean(xbar5), mean(xbar20))
Std.Dev <- c(sd(cms$MORT.30.AMI.Performance.Rate, na.rm = T), sd(xbar5), sd(xbar20))
data.frame(Set, Mean, Std.Dev)

Set,Mean,Std.Dev
Population,0.8653658,0.010505600
Sample Size 5,0.8653003,0.004627252
Sample Size 20,0.8653550,0.002378530


___
### Comparison of Sampling Methods

While this data set is small enough that I can perform analysis on the population, it is important to establish the effect of various sampling methods, should, for example, I analyze quality metrics for outpatient facilities as well, or expand the hospital set to include countries outside the US. I will be comparing the population to samples of size 200 using the following 3 methods, again focusing on the mortality rate of Acute Myocardial Infarction:

- Simple random sampling without replacement
- Systematic sampling
- Stratified sampling (based on State)

In [69]:
# Set Seed, Create Vectors for Mean & Standard Deviation
set.seed(123)
Mean <- numeric(3)
Std.Dev <- numeric(3)

# Simple Random Sampling Without Replacement
s <- srswor(200, length(cms$MORT.30.AMI.Performance.Rate))
rows <- (1:length(cms$MORT.30.AMI.Performance.Rate))[s != 0]
simprand <- cms[rows, "MORT.30.AMI.Performance.Rate"]
Mean[1] <- mean(cms[rows, "MORT.30.AMI.Performance.Rate"], na.rm = T)
Std.Dev[1] <- sd(cms[rows, "MORT.30.AMI.Performance.Rate"], na.rm = T)

# Systematic Sampling
N <- length(cms$MORT.30.AMI.Performance.Rate)
n <- 200
k <- N/n
r <- sample(k, 1)
rows <- round(seq(r, by = k, length = n))
systematic <- cms[rows, "MORT.30.AMI.Performance.Rate"]
Mean[2] <- mean(cms[rows, "MORT.30.AMI.Performance.Rate"], na.rm = T)
Std.Dev[2] <- sd(cms[rows, "MORT.30.AMI.Performance.Rate"], na.rm = T)

# Stratified Sampling
order.index <- order(cms$State)
cmsord <- cms[order.index, ]
freq <- table(cms$State)
st.sizes <- 200 * freq/sum(freq)
zerostrata <- st.sizes[floor(st.sizes) == 0]
cmsnozero <- cmsord[!cmsord$State %in% names(zerostrata), ]
freq <- table(cmsnozero$State)
st.sizes <- 200 * freq/sum(freq)
st <- strata(cmsnozero, stratanames = c("State"), size = st.sizes, method = "srswor", 
    description = F)
st.sample <- getdata(cmsnozero, st)
strat <- st.sample$MORT.30.AMI.Performance.Rate
Mean[3] <- mean(st.sample$MORT.30.AMI.Performance.Rate, na.rm = T)
Std.Dev[3] <- sd(st.sample$MORT.30.AMI.Performance.Rate, na.rm = T)

# Plot
api_create(plot_ly(width = 800, height = 600, alpha = 0.6) %>% add_histogram(x = ~simprand, 
    name = "Simple Random") %>% add_histogram(x = ~systematic, name = "Systematic") %>% 
    add_histogram(x = ~strat, name = "Stratified") %>% layout(barmode = "overlay", 
    title = "Comparison of Sampling Methods - Distribution of Values", xaxis = list(title = ""), 
    margin = list(l = 50, r = 50, b = 50, t = 70)), filename = "sampmethist")

<div>
    <a href="https://plot.ly/~djzero/12/?share_key=UmF7gmMh2q6XsygGM2vU0W" target="_blank" title="sampmethist" style="display: block; text-align: center;"><img src="https://plot.ly/~djzero/12.png?share_key=UmF7gmMh2q6XsygGM2vU0W" alt="sampmethist" style="max-width: 100%;width: 800px;"  width="800" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="djzero:12" sharekey-plotly="UmF7gmMh2q6XsygGM2vU0W" src="https://plot.ly/embed.js" async></script>
</div>
The overlaid histogram seems to indicate that for this particular seed value, there is a large amount of overlap in the values from all 3 sampling methods, with the Stratified sample showing more of a tendency toward the mean. 

With such high overlap, a table is necessary to determine which sampling method best reflects the distribution of the population:

In [67]:
# Create Dataframe
Samp.Type <- c("Simple Random", "Systematic", "Stratified")
output <- data.frame(Samp.Type, Mean, Std.Dev)
output["Mean.Var"] <- abs(output["Mean"] - mean(cms[, "MORT.30.AMI.Performance.Rate"], 
    na.rm = T))
output["Std.Dev.Var"] <- abs(output["Std.Dev"] - sd(cms[, "MORT.30.AMI.Performance.Rate"], 
    na.rm = T))
output[, c("Samp.Type", "Mean", "Mean.Var", "Std.Dev", "Std.Dev.Var")]

Samp.Type,Mean,Mean.Var,Std.Dev,Std.Dev.Var
Simple Random,0.8645573,0.0008084721,0.010663712,0.0001581120
Systematic,0.8659771,0.0006113556,0.011084306,0.0005787058
Stratified,0.8652112,0.0001545761,0.009537537,0.0009680638


The table reinforces the conclusion from viewing the histogram that the Stratified sampling method resulted in the lowest standard deviation of the three. Looking at the absolute value of the difference between each method and the population, we see that the mean Performance Rate from the Stratified sample is the closest to the mean Performance Rate for the population, with only ~2 basis points of difference. However, the Stratified sample is least likely to reflect the spread of the population, with a standard deviation ~10 basis points different. Instead, the spread is best reflected in the Simple Random sample, with ~2 basis points difference between its standard deviation and that of the population.